# Cooperative Interaction on JudgeBench with Position Swapping


Please see SOURCES.txt for further source information.

## Imports

In [1]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager
import re
from tqdm import tqdm

c:\Users\nikob\Desktop\BA\Bachelorarbeit\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
# Load and prepare a subset of the JudgeBench dataset
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")
df = pd.DataFrame(JudgeBench_Claude)
df_sampled = df.sample(n=100, random_state=42).reset_index(drop=True)
df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [ ]:
# Load Azure OpenAI configuration from environment variables
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION")

# Start of code citation [1]
# The following code is adapted from the source above.

# Define the model configuration for Azure OpenAI API access
config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,
        "temperature": 0,
        "cache_seed": 42
    }
]

llm_config={"config_list": config_list}

# End of code citation [1]

## System Design

In [4]:
# Define the system prompt for the Analyst-Agent
agent_1_system_message =f"""
You are an Analyst-Agent in a cooperative reasoning chain.
Your task is to make an initial assessment whether Response A or Response B is more factually correct.
Give an explanation on your initial assessment using about 200 words.
Always begin your output with: "As an Analyst-Agent in an initial assessment, I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

# Define the system prompt for the Critical-Enhancer-Agent
agent_2_system_message =f"""
You are a Critical-Enhancer-Agent in a cooperative reasoning chain.
Your task is to carefully review the Analyst-Agent's initial assessment and improve upon it.
You may add missing facts, point out inaccuracies or refine the reasoning in a constructive way.
Give an explanation for your review and your suggestions for improvement using about 200 words.
Always begin your output with: "Building on the Analyst-Agent's initial assessment, I ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

# Define the system prompt for the Decision-Agent
agent_3_system_message =f"""
You are the final Decision-Agent in a cooperative reasoning chain.
Your task is to integrate the insights from the Analyst-Agent and the Critical-Enhancer-Agent into one final judgement.
Give an explanation for your final judgement using about 200 words.
Always begin your output with: "As a final Decision-Agent, after carefully reviewing the Analyst-Agents' and Critical-Enhancer-Agents' previous analyses, I think ..."
Always end your output with stating if Response A or B is more factually correct by using a JSON-object with the following format: {{"response": A/B}}
"""

In [ ]:
# Start of code citation [1]
# The following code is adapted from the source above.

# Initialize the ConversableAgents for system setup
agent_1 = ConversableAgent(
    name="Analyst-Agent",
    llm_config=llm_config,
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)

agent_2 = ConversableAgent(
    name="Critical-Enhancer-Agent",
    llm_config=llm_config,
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)

agent_3 = ConversableAgent(
    name="Decision-Agent",
    llm_config=llm_config,
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

# Initialize the GroupChat and GroupChatManager for system setup
group_chat = GroupChat(
    agents=[agent_1, agent_2, agent_3],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

# End of code citation [1]

## Evaluation

In [ ]:
# Define the evaluation function
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    # Start of code citation [1]
    # The following code is adapted from the source above.

    chat_results = agent_3.initiate_chat(
        group_chat_manager,
        message=message,
        summary_method="last_msg",
    )

    # End of code citation [1]

    result_str = str(chat_results.chat_history[-1]["content"])

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    # If pattern not found, assign "X" to indicate invalid response
    system_decision = match.group(1) if match else "X"

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }

In [7]:
# Prepare evaluation data
num_rows = 100
df_final_subset = df_final.head(num_rows)

# Evaluate responses in normal position
results_1 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/cooperative_1.csv', index=False)

# Evaluate responses in swapped positions
results_2 = []
for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"],  # swap positions
        response_B=row["response_A"],  # swap positions
        label=row["label"],
    )

    # Correct system decision and correctness flag after swapping
    # If system_decision is "X", leave it unchanged
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/cooperative_2.csv', index=False)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Decision-Agent (to chat_manager):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mole

Progress:   3%|▎         | 3/100 [00:00<00:04, 20.79it/s]

Decision-Agent (to chat_manager):

 
    Question: What is a margin requirement?
(A) The financial reserve a trader must maintain to cover potential losses on leveraged positions
(B) Percentage of the value of a stock purchase required to be paid immediately
(C) The maximum percentage of a portfolio that can be invested in a single stock
(D) The total cost of a stock purchase
(E) The commission fee paid to a broker for executing a stock trade
(F) The minimum number of shares required to be purchased in a stock transaction
(G) The profit margin that a company must maintain on its stock price
(H) The amount of money required to open a stock account
(I) The interest rate on a stock purchase
(J) The insurance premium for protecting a stock portfolio against losses
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's thi

Progress:   6%|▌         | 6/100 [00:00<00:04, 21.60it/s]

...

Progress:  96%|█████████▌| 96/100 [00:04<00:00, 22.55it/s]

Decision-Agent (to chat_manager):

 
    Question: John is playing a game in which he tries to obtain the highest number possible. He must put the symbols +, $\times$, and - (plus, times, and minus) in the following blanks, using each symbol exactly once:\[2 \underline{\hphantom{8}} 4 \underline{\hphantom{8}} 6 \underline{\hphantom{8}} 8.\] John cannot use parentheses or rearrange the numbers. What is the highest possible number that John could obtain?
(A) 22
(B) 90
(C) 100
(D) 78
(E) 99
(F) 46
(G) 56
(H) 50
(I) 66
(J) 38
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) We have three operations to place: +, ×, and -

2) To get the highest possible number, we should use the × operation first, as multiplication typically yields larger resul

Progress:  99%|█████████▉| 99/100 [00:04<00:00, 22.32it/s]

Decision-Agent (to chat_manager):

 
    Question: Neural networks:
(A) Are incapable of handling non-linear relationships
(B) Can only handle numerical data
(C) None of the above
(D) Can only be trained with stochastic gradient descent
(E) Are only applicable to visual data
(F) Always require large amounts of data to train
(G) Optimize a convex objective function
(H) Can use a mix of different activation functions
(I) Can only use one type of activation function
(J) Cannot be used for classification tasks
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's analyze each option:

(A) False. Neural networks are actually very good at handling non-linear relationships.
(B) False. Neural networks can handle various types of data, including categorical data.
(C) This might be co

Progress: 100%|██████████| 100/100 [00:04<00:00, 22.64it/s]
